In [1]:
import cobra
import pandas as pd
from collections import OrderedDict

In [2]:
def build_reaction_equation_from_metabolites_dict(met_dict, arrow='<=>', floatdecimal=6):
    lhs = []; rhs = [];
    for k,v in met_dict.items():
        v = float(v)
        if v == -1:
            lhs.append(k)
        elif v == 1:
            rhs.append(k)
        elif v < 0 and v != -1 and v.is_integer():
            lhs.append(' '.join([str(-int(v)), k]))
        elif v > 0 and v != 1 and v.is_integer():
            rhs.append(' '.join([str(int(v)), k]))
        elif v < 0 and v != -1:
            lhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(-v), k]))
        elif v > 0 and v != 1:
            rhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(v), k]))
    return ' '.join([ ' + '.join(lhs), arrow, ' + '.join(rhs)])

In [3]:
df_data = pd.read_excel('./phenotype_data_collection_2022-05-06.xlsx', sheet_name='Data')
idx = range(0,72)
df_data = df_data.loc[idx, :]
df_data.index = df_data.Dataset.to_list()

In [4]:
model = cobra.io.load_json_model('../../build_model/input/precursors/GSM_iSace1144.json')
model.solver = 'cplex'
model.reactions.EX_glc__D_e.bounds = (0,1000)
model.reactions.EX_o2_e.bounds = (-1000,1000)
model.reactions.ATPM_c.bounds = (0,1000)
model.reactions.HCO3E_e.bounds = (0,0)

# Added on 2022-May-06
model.reactions.FDH_c.knock_out()
#model.reactions.ABTA_c.knock_out()
model.reactions.get_by_id('4ABTORy_c').knock_out()
model.reactions.get_by_id('4ABTORx_c').knock_out()
model.reactions.FADH2t_c_m.knock_out()

# Add NADH oxidase reaction
rxn = cobra.Reaction('NADHOX_c')
model.add_reaction(rxn)
rxn.reaction = 'nadh_c + o2_c + h_c --> nad_c + h2o2_c'
model.reactions.NADHOX_c.bounds = (0,0)

model.objective = dict()
model.reactions.ATPM_c.objective_coefficient = 1
default_bounds = {rxn.id:rxn.bounds for rxn in model.reactions}

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2023-07-30


In [5]:
# Remove GAM from biomass equation
for biom in ['BIOMASS_AERO_SC_hvd', 'BIOMASS_BATCHANAERO_SC_hvd']:
    adp = model.metabolites.adp_c
    gam0 = model.reactions.get_by_id(biom).metabolites[adp]
    met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
    for met in ['atp_c', 'h2o_c']:
        met_dict[met] += gam0
    for met in ['adp_c', 'pi_c', 'h_c']:
        met_dict[met] -= gam0
    biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
    model.reactions.get_by_id(biom).reaction = biom_eqn

In [6]:
for i in df_data.index:
    if df_data.Remove_tag[i] == 'Removed':
        print('', '')
        continue
    rxns_tuned = []
    
    # Implement oxygen conditions. If anaerobic, supplement nutrient
    if df_data.Oxy[i] == 0:
        model.reactions.EX_o2_e.bounds = (0,1000)
        rxns_supp = ['EX_ergst_e', 'EX_hdcea_e', 'EX_ocdcea_e', 'EX_nac_e', 'EX_pnto__R_e']
        rxns_tuned += rxns_supp
        for r in rxns_supp:
            model.reactions.get_by_id(r).bounds = (-1000,1000)
    else:
        model.reactions.EX_o2_e.bounds = (-1000,1000)
            
    # Select biomass reaction and implement growth rate
    gr = df_data.Growth_rate[i]
    rxns_tuned += ['BIOMASS_BATCHANAERO_SC_hvd', 'compCERANAERO_r', 'BIOMASS_AERO_SC_hvd']
    if df_data.Oxy[i] == 0:
        model.reactions.BIOMASS_BATCHANAERO_SC_hvd.bounds = (gr,gr)
        model.reactions.compCERANAERO_r.bounds = (0,1000)
    else:
        model.reactions.BIOMASS_AERO_SC_hvd.bounds = (gr,gr)
        model.reactions.BIOMASS_BATCHANAERO_SC_hvd.bounds = (0,0)
        model.reactions.compCERANAERO_r.bounds = (0,0)
        
    # Implement substrate uptake
    sub_upt = df_data.Substrate_uptake[i].split(' | ')
    for s in sub_upt:
        r,ctype,v = s.split(':')
        rxns_tuned.append(r)
        v = float(v)
        if ctype == 'E':
            model.reactions.get_by_id(r).bounds = (-v,-v)
        elif ctype == 'U':
            model.reactions.get_by_id(r).bounds = (-v,1000)
        else:
            print('There is a typo in ' + s)
            
    # Implement oxygen uptake
    o2_upt = df_data.Oxy_uptake[i]
    if pd.isnull(o2_upt) == False:
        r,ctype,v = o2_upt.split(':')
        v = float(v)
        if ctype == 'E':
            model.reactions.get_by_id(r).bounds = (-v,-v)
        elif ctype == 'U':
            model.reactions.get_by_id(r).bounds = (-v,1000)
        else:
            print('There is a typo in ' + s)
            
    # Implement secretions
    secs = df_data.Secretions[i]
    if pd.isnull(secs) == False:
        secs = secs.split(' | ')
        for s in secs:
            r,ctype,v = s.split(':')
            rxns_tuned.append(r)
            v = float(v)
            if ctype == 'E':
                model.reactions.get_by_id(r).bounds = (v,v)
            elif ctype == 'L':
                model.reactions.get_by_id(r).bounds = (v,1000)
            else:
                print('There is a typo in ' + s)
            
    # Implement notes
    if df_data.Simulation_settings[i] == 'Add_NADH_oxidase':
        rxns_tuned.append('NADHOX_c')
        model.reactions.NADHOX_c.bounds = (0,1000)
    elif df_data.Simulation_settings[i] == 'Disable_NH4_uptake':
        rxns_tuned.append('EX_nh4_e')
        model.reactions.EX_nh4_e.bounds = (0,1000)
    else:
        if pd.isnull(df_data.Simulation_settings[i]) == False:
            s = df_data.Simulation_settings[i]
            r,ctype,v = s.split(':')
            v = float(v)
            if ctype == 'E':
                model.reactions.get_by_id(r).bounds = (v,v)
            elif ctype == 'L':
                model.reactions.get_by_id(r).lower_bound = v
            elif ctype == 'U':
                model.reactions.get_by_id(r).upper_bound = v
            else:
                print('There is a typo in ' + s)
        
    # Implement mean value fix assuming experimental error are roughly 10% of mean value
    adjs = df_data.Mean_value_adjustment[i]
    if pd.isnull(adjs) == False:
        adjs = adjs.split(' | ')
        for adj in adjs:
            r,var,val = adj.split(':')
            val = float(val)
            if var == 'LB':
                model.reactions.get_by_id(r).lower_bound = val
            elif var == 'UB':
                model.reactions.get_by_id(r).upper_bound = val
            else:
                print('There is a typo in ' + adj)
            
    # FBA maximizing ATPM
    fba = model.optimize()
    print(fba.status, round(fba['ATPM_c'],4))
    
    for r in rxns_tuned:
        model.reactions.get_by_id(r).bounds = default_bounds[r]

optimal 21.0172
optimal 22.0802
optimal 29.667
optimal 29.6629
optimal 54.1876
optimal 21.2544
optimal 9.0822
optimal 9.9543
optimal 9.5948
optimal 10.3971
optimal 41.0776
optimal 32.8522
optimal 19.9184
optimal 13.0813
optimal 9.4109
optimal 18.2094
optimal 7.6114
optimal 31.6028
optimal 30.779
optimal 8.6629
optimal 25.0541
optimal 18.7486
optimal 3.2834
optimal 3.6834
optimal 7.0567
optimal 10.4008
optimal 10.6967
optimal 8.7851
optimal 2.5176
optimal 5.0737
optimal 14.0932
optimal 7.8798
optimal 9.7143
optimal 8.238
optimal 7.5794
optimal 6.8081
optimal 8.2436
optimal 49.1612
optimal 19.2896
optimal 14.6805
optimal 22.343
optimal 28.8511
optimal 30.8368
optimal 11.4724
optimal 18.1337
optimal 14.1446
optimal 19.3713
optimal 29.3384
optimal 34.1
optimal 44.1459
optimal 59.4451
optimal 7.7354
optimal 7.47
optimal 13.2808
optimal 18.1493
optimal 20.6076
optimal 38.4029
optimal 61.3573
optimal 16.5051
optimal 13.9609
optimal 3.0098
optimal 17.9128
optimal 17.766
optimal 25.9818
optimal